In [ ]:
!pip3 install torchinfo

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import torchinfo
import matplotlib.pyplot as plt

## Patching + linear_embed

In [ ]:
class Patching(nn.Module):
    """
    Input: Images (B,C,H,W)
    Output: (B,N+1,d_model)
    """
    def __init__(self,C,patch_size=16,d_model=786):
        super().__init__()
        self.patch_size=patch_size
        self.C=C
        self.in_=int((patch_size**2)*self.C)
        self.linear=nn.Linear(in_features=self.in_,out_features=d_model)

    def forward(self,img):
        B,C,H,W=img.shape
        patches=img.unfold(2,self.patch_size,self.patch_size).unfold(3,self.patch_size,self.patch_size) # (B,C, num_patches_H, num_patches_W, patch_size, patch_size)
        patches=patches.permute(0,2,3,1,4,5).reshape(B,-1,self.in_)
        #N,P**2C

        return self.linear(patches) # B,N,d_model



In [ ]:
Patch_test=Patching(C=3,patch_size=16,d_model=768)
img_test=torch.randn((10,3,224,224))
Patch_test(img_test).shape

In [ ]:
class ViT_test(nn.Module):
    def __init__(self,H,W,C,patch_size=16,d_model=786):
        super().__init__()
        self.C=C
        self.patch_size=patch_size
        self.num_patches=int((H*W)/(self.patch_size**2))
        self.d_model=d_model
        self.pos_embed=nn.Parameter(torch.randn(1,self.num_patches+1,d_model))
        self.cls_token=nn.Parameter(torch.zeros(1,d_model)) #1,d_model
        self.patching=Patching(C=self.C,patch_size=self.patch_size,d_model=self.d_model)
    def forward(self,imgs):
        imgs=self.patching(imgs) #B,N,d_model
        cls_token=self.cls_token.expand(imgs.shape[0],-1,-1)
        imgs=torch.cat((cls_token,imgs),dim=1) # B,N+1,d_model
        imgs=imgs + self.pos_embed
        return imgs




In [ ]:
vit_test=ViT_test(H=224,W=224,C=3)
vit_test(img_test).shape # B,num_patches+1,d_model


## Transformer block

In [ ]:
import math
import copy
import torch.nn.functional as F
import torch.optim as optim


device="cuda" if torch.cuda.is_available() else "cpu"
def clones(module,N):
    """
    Create a list of N identical layers.

    Args:
        module (nn.Module): A neural network module to be cloned.
        N (int): The number of clones to create.

    Returns:
        nn.ModuleList: A list containing N deep copies of the input module.
    """
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

def attention(query,key,value,dropout=0):


  #query, key, value -> N,h,T,d_k
  d_k=query.size(-1)
  scores=torch.matmul(query,key.transpose(-1,-2))/math.sqrt(d_k) # N,h,T,d_k @ N,h,d_k,T = N,h,T,T

  p_attn=scores.softmax(dim=-1)
  if dropout is not None:
    p_attn=dropout(p_attn)

  return torch.matmul(p_attn,value), p_attn


class MultiHeadedAttention(nn.Module):
  def __init__(self,h,d_model,dropout=0):
    """
    Create a MultiHeadedAttention layer.

    Args:
        h (int): The number of heads in the multi-head attention mechanism.
        d_model (int): The number of expected features in the input.
        dropout (float, optional): The dropout to apply to the attention weights. Defaults to 0.1.
    """

    super(MultiHeadedAttention,self).__init__()

    self.d_k=d_model//h
    self.h=h
    self.linears=clones(nn.Linear(d_model, d_model), 4)
    self.dropout=nn.Dropout(p=dropout)

  def forward(self,query,key,value):


    """
    Compute the forward pass of the multi-headed attention layer.

    Args:
        query (torch.Tensor): The query tensor.
        key (torch.Tensor): The key tensor.
        value (torch.Tensor): The value tensor.


    Returns:
        torch.Tensor: The output of the forward pass.
    """


    nbatches=query.shape[0]

    query,key,value=[
        lin(x).view(nbatches,-1,self.h,self.d_k).transpose(1,2)
        for lin,x in zip(self.linears,(query,key,value))
    ]

    x,self_attn=attention(query,key,value,dropout=self.dropout)

    x=(x.transpose(1,2).contiguous().view(nbatches,-1,self.h*self.d_k))

    del query
    del key
    del value

    return self.linears[-1](x)

### MLP

In [ ]:
class MLP(nn.Module):
    def __init__(self,d_model,dropout=0):
        super().__init__()
        self.dropout=nn.Dropout(p=dropout)
        self.linear1=nn.Linear(in_features=d_model,out_features=4*d_model)
        self.linear2=nn.Linear(in_features=4*d_model,out_features=d_model)

    def forward(self,x):
        x=self.linear1(x)
        x=self.dropout(F.gelu(x))
        x=self.linear2(x)
        x=self.dropout(x)
        return x


In [ ]:
mlp_test = MLP(d_model=768, dropout=0.1)

x_test = torch.randn(10, 197, 768)  # B=10, N=197, D=768

out = mlp_test(x_test)
print(out.shape)

### residual connection

In [ ]:
class SubLayerConnection(nn.Module):
  def __init__(self,d_model,dropout):
    """
  Initialize a sublayer connection layer.

  Parameters:
  d_model (int): The number of expected features in the input.
  dropout (float): The amount of dropout to apply.
    """
    super(SubLayerConnection,self).__init__()
    self.norm=nn.LayerNorm(d_model)
    self.dropout=nn.Dropout(dropout)

  def forward(self,x,sublayer):

    return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class ViT_layer(nn.Module):

  def __init__(self,self_attn,mlp,dropout,hidden=768*4,h=12,d_model=768):

    """
  Initialize a ViT_layer

  Parameters:
  self_attn (nn.Module): A multi-headed self-attention layer.
  feed_forward (nn.Module): A feed-forward layer.
  dropout (float): The amount of dropout to apply.
  hidden (int): The number of neurons in the hidden layer of the feed-forward layer. Defaults to 768*4.
  h (int): The number of attention heads. Defaults to 12.
  d_model (int): The number of expected features in the input. Defaults to 768.
  """
    super().__init__()
    self.self_attn=self_attn
    self.h=h
    self.mlp=mlp
    self.sublayer=clones(SubLayerConnection(d_model,dropout),2)
    self.d_model=d_model


  def forward(self,x):

    x=self.sublayer[0](x, lambda x:self.self_attn(x,x,x))
    return self.sublayer[1](x, self.mlp)



In [ ]:
layer_test=ViT_layer(self_attn=MultiHeadedAttention(h=12,d_model=768),mlp=MLP(d_model=768),dropout=0.1)
layer_test(x_test).shape

### Transformer

In [ ]:
class ViT(nn.Module):
    def __init__(self,n_classes=1000,H=224,W=224,C=3,patch_size=16,d_model=768):
        super().__init__()
        self.H=H
        self.W=W
        self.C=C
        self.patch_size=patch_size
        self.d_model=d_model
        self.num_patches=int((H*W)/(self.patch_size**2))
        self.pos_embed=nn.Parameter(torch.randn(1,self.num_patches+1,d_model))
        self.cls_token=nn.Parameter(torch.zeros(1,d_model))
        self.patching=Patching(C=self.C,patch_size=self.patch_size,d_model=self.d_model)
        self.layers=clones(ViT_layer(self_attn=MultiHeadedAttention(h=12,d_model=self.d_model),mlp=MLP(d_model=self.d_model),dropout=0.1),12)
        self.norm=nn.LayerNorm(self.d_model)
        self.mlp_head=nn.Linear(in_features=self.d_model,out_features=n_classes)

    def forward(self,x):
        x=self.patching(x)
        x=torch.cat((self.cls_token.expand(x.shape[0],-1,-1),x),dim=1)
        x=x+self.pos_embed
        for layer in self.layers:
            x=layer(x)
        x=self.norm(x)
        cls_token_final=x[:,0] #batch_size,d_model
        x=self.mlp_head(cls_token_final) #batch_size,n_classes
        return x,cls_token_final #batch_size,n_classes batch_size,d_model

In [ ]:
model_test = ViT(
    n_classes=1000,
    H=224,
    W=224,
    C=3,
    patch_size=16,
    d_model=768
)

# Random test input: batch of 8 RGB images of size 224x224
x_test = torch.randn(8, 3, 224, 224)

# Forward pass
logits, cls_token_output = model_test(x_test)

# Print shapes to verify
print("Logits shape:", logits.shape)               # Expected: [8, 1000]
print("CLS Token Output shape:", cls_token_output.shape)  # Expected: [8, 768]

In [ ]:
x_test=torch.randn(4,3,224,224)
logits, cls_token_output=model_test(x_test)
print("Logits shape:", logits.shape)
print("CLS Token Output shape:", cls_token_output.shape)

## Data transformations

In [ ]:
from torchvision import transforms
from torchvision.transforms import v2
from torchvision.transforms.functional import InterpolationMode

In [ ]:
#Imagenet statistics
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

In [ ]:
v2.RandomResizedCrop

In [ ]:
def train_transformatons(image_size=(224,224),image_mean=IMAGENET_MEAN,image_std=IMAGENET_STD
                         , hflip_probab=0.5, interpolation=InterpolationMode.BILINEAR, random_aug_magnitude=9
                         ):
    #interpolation: How to resize image when cropping/resizing. BILINEAR is smooth and preferred for ViTs.
    transformation_chain=[]

    transformation_chain.append(
        v2.RandomResizedCrop(image_size,interpolation=interpolation,antialias=True)
        )

    if hflip_probab > 0:
        transformation_chain.append(v2.RandomHorizontalFlip(p=hflip_probab)) #flips image horizontaly with 50% probab

    if random_aug_magnitude > 0:
        print("Enabling Random Augmentation!")
        transformation_chain.append(v2.RandAugment(magnitude=random_aug_magnitude, interpolation=interpolation))
        #Applies random transformations (e.g., brightness, contrast, shear, rotation).
        #	Magnitude controls how strong the augmentations are.
        #Makes your dataset more diverse → better generalization.


    transformation_chain.append(v2.PILToTensor()) #Converts from PIL Image to PyTorch Tensor.
    transformation_chain.append(v2.ToDtype(torch.float32,scale=True)) #scales pixels values from [0,255] to [0.0,1.0]
    transformation_chain.append(v2.Normalize(mean=image_mean,std=image_std)) # pixel-mean/std
    #brings each channel to zero-mean, unit variance. Essential for fast training and stability

    return transforms.Compose(transformation_chain)





In [ ]:
def eval_transformations(image_size=(224,224), resize_image=(256,256), image_mean=IMAGENET_MEAN, image_std=IMAGENET_STD,
                         interpolation=InterpolationMode.BILINEAR
                         ):

    #meant to be used during evaluation or inference, not training
    transformations=transforms.Compose([
        v2.Resize(resize_image,interpolation=interpolation, antialias=True),
        v2.CenterCrop(image_size),
        v2.PILToTensor(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=image_mean, std=image_std)
    ])

    return transformations


In [ ]:
from torch.utils.data import default_collate
# customizing the collat_fn of the dataloader
#blend images after theyve been batched together
#but before sending them to the model
def mixup_cutmix_collate_fn(mixup_alpha=0.2, cutmix_alpha=1.0, num_classes=1000):
    #returns a custom collate fn for the dataloader
    #mixup_alpha: the probability of applying mixup
    #cutmix_alpha: the probability of applying cutmix
    #num_classes: the number of classes in the dataset

    mix_cut_transforms=None
    mixup_cutmix=[]

    if mixup_alpha > 0:
        print("Enabling Mixup!")
        mixup_cutmix.append(v2.MixUp(alpha=mixup_alpha,num_classes=num_classes))

    if cutmix_alpha > 0:
        print("Enabling Cutmix!")
        mixup_cutmix.append(v2.CutMix(alpha=cutmix_alpha,num_classes=num_classes))

    if len(mixup_cutmix) > 0:
        mix_cut_transforms=v2.RandomChoice(mixup_cutmix)

    def collate_fn(batch):
        collated=default_collate(batch)

        if mix_cut_transforms is not None:
            collated=mix_cut_transforms(collated)

        return collated

    return collate_fn

In [ ]:
!pip3 install datasets

In [ ]:
from datasets import load_dataset

ds = load_dataset("zh-plus/tiny-imagenet")
ds

In [ ]:
train_ds=ds["train"].select(range(7000))
val_ds=ds["valid"].select(range(1000))
print(len(train_ds))
print(train_ds)

In [ ]:
train_ds["image"][0]

In [ ]:
from torch.utils.data import DataLoader
class Imagenet_Wrapper(torch.utils.data.Dataset):
  def __init__(self,hf_dataset,transform=None):
    self.dataset=hf_dataset
    self.transform=transform

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self,idx):
    image=self.dataset["image"][idx]
    label=self.dataset["label"][idx]

    if self.transform:
      image=self.transform(image)

    return image,label


train_tfms=train_transformatons()
eval_tfms=eval_transformations()
wrapped_train_ds=Imagenet_Wrapper(train_ds,transform=train_tfms)
wrapped_val_ds=Imagenet_Wrapper(val_ds,transform=eval_tfms)


collate_fn=mixup_cutmix_collate_fn(num_classes=200)
train_loader=DataLoader(dataset=wrapped_train_ds,
                        batch_size=32,
                        collate_fn=collate_fn,
                        shuffle=True
                        )

val_loader=DataLoader(dataset=wrapped_val_ds,
                      batch_size=32,
                      collate_fn=collate_fn

                      )

sample_train=next(iter(train_loader))
sample_val=next(iter(val_loader))
print('sample_train',sample_train[0].shape)
print('sample_val',sample_val[0].shape)

print(sample_train)

In [ ]:
print(len(wrapped_train_ds))

In [ ]:
pred = torch.tensor([
    [3, 0, 4, 2, 5],   # Sample 1
    [1, 2, 4, 5, 0],   # Sample 2
    [4, 0, 5, 2, 1]    # Sample 3
])  # Shape: [3, 5]

pred.transpose(0,1)

In [ ]:
def accuracy(output,target,topk=(1,5)):
  #output -> (batch_size, num_classes)
  #target ->
    # can be hard labels (batch_size,)
    #can be soft labels (mixup,cutmix) (batch_size,num_classes)



  with torch.inference_mode(): #turn of gradient tracking -> faster + memory efficient
    maxk=max(topk)

    batch_size=target.size(0)

    if target.ndim==2: #if soft labels
      target=target.max(dim=1)[1] #argmax along each row = the dominant class

    values,pred=output.topk(maxk,dim=-1,largest=True,sorted=True)
    #pred -> batch_size,maxk

    pred=pred.transpose(0,1)
    #maxk,batchsize

    correct = (pred==target) # target is broadcasted to desired shape


    accs = []
    for k in topk:
        correct_k = correct[:k].flatten().sum(dtype=torch.float32)
        accs.append((correct_k / batch_size))
    if len(accs) == 1:
        return accs[0]
    else:
        return accs

In [ ]:
output = torch.tensor([
    [0.1, 0.2, 3.0, 0.4],  # GT = 2 (correct in top-1)
    [1.0, 0.9, 0.8, 0.7],  # GT = 0 (correct in top-1)
    [0.2, 3.5, 0.1, 0.9],  # GT = 1 (correct in top-1)
])

target = torch.tensor([2, 0, 1])

acc = accuracy(output, target, topk=(1, 2))
print("Top-1 and Top-2 Acc (hard labels):", acc)  # Expected: [1.0, 1.0]

In [ ]:
output = torch.tensor([
    [0.1, 0.2, 3.0, 0.4],  # GT = 2 (should still be counted as class 2)
    [1.0, 0.9, 0.8, 0.7],  # GT = 0
    [0.2, 3.5, 0.1, 0.9],  # GT = 1
])

target = torch.tensor([
    [0.0, 0.0, 1.0, 0.0],  # class 2
    [1.0, 0.0, 0.0, 0.0],  # class 0
    [0.0, 1.0, 0.0, 0.0],  # class 1
])

acc = accuracy(output, target, topk=(1, 2))
print("Top-1 and Top-2 Acc (soft labels):", acc)  # Expected: [1.0, 1.0]

In [ ]:
output = torch.tensor([
    [0.1, 0.2, 3.0, 0.4],  # GT = 0 → wrong
    [1.0, 0.9, 0.8, 0.7],  # GT = 0 → correct
    [0.2, 3.5, 0.1, 0.9],  # GT = 3 → not top-1, but in top-2
])

target = torch.tensor([0, 0, 3])

acc = accuracy(output, target, topk=(1, 2))
print("Top-1 and Top-2 Acc (partial miss):", acc)  # Expected: something like [0.33, 0.66]

# train and val

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.parallel import DataParallel
from tqdm import tqdm

# Initialize model with DataParallel
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ViT(n_classes=200).to(device)
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = DataParallel(model)

# Loss function (handles both regular and mixup/cutmix labels)
def loss_fn(outputs, targets):
    if targets.ndim == 2:  # Soft labels (mixup/cutmix)
        return -torch.mean(torch.sum(F.log_softmax(outputs, dim=1) * targets, dim=1))
    else:  # Hard labels
        return F.cross_entropy(outputs, targets)

# Optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.05)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader)*10)

def train_epoch(model, train_loader, optimizer, epoch):
    model.train()
    total_loss = 0
    top1_acc = 0
    top5_acc = 0
    
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]", leave=False)
    for batch_idx, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Forward pass
        outputs, _ = model(inputs)
        loss = loss_fn(outputs, targets)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        acc1, acc5 = accuracy(outputs, targets, topk=(1, 5))
        top1_acc += acc1.item()
        top5_acc += acc5.item()
        total_loss += loss.item()
        
        # Update progress bar
        progress_bar.set_postfix({
            'loss': loss.item(),
            'top1': acc1.item(),
            'top5': acc5.item()
        })
    
    scheduler.step()
    
    avg_loss = total_loss / len(train_loader)
    avg_top1 = top1_acc / len(train_loader)
    avg_top5 = top5_acc / len(train_loader)
    
    return avg_loss, avg_top1, avg_top5

def validate(model, val_loader):
    model.eval()
    total_loss = 0
    top1_acc = 0
    top5_acc = 0
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc="[Validation]", leave=False)
        for inputs, targets in progress_bar:
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs, _ = model(inputs)
            loss = loss_fn(outputs, targets)
            
            acc1, acc5 = accuracy(outputs, targets, topk=(1, 5))
            top1_acc += acc1.item()
            top5_acc += acc5.item()
            total_loss += loss.item()
            
            progress_bar.set_postfix({
                'val_loss': loss.item(),
                'val_top1': acc1.item(),
                'val_top5': acc5.item()
            })
    
    avg_loss = total_loss / len(val_loader)
    avg_top1 = top1_acc / len(val_loader)
    avg_top5 = top5_acc / len(val_loader)
    
    return avg_loss, avg_top1, avg_top5

# Main training loop
num_epochs = 2
best_val_acc = 0.0

for epoch in range(num_epochs):
    train_loss, train_top1, train_top5 = train_epoch(model, train_loader, optimizer, epoch)
    val_loss, val_top1, val_top5 = validate(model, val_loader)
    
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Top1: {train_top1:.4f} | Top5: {train_top5:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Top1: {val_top1:.4f} | Top5: {val_top5:.4f}")
    
    # Save best model
    if val_top1 > best_val_acc:
        best_val_acc = val_top1
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.module.state_dict() if isinstance(model, DataParallel) else model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_top1,
        }, 'best_vit_model.pth')
    
    print(f"Best Val Acc: {best_val_acc:.4f}")

In [ ]:
train_loss, train_top1, train_top5

In [ ]:
val_loss, val_top1, val_top5